# 📓 03 Geoestadística y Análisis de Autocorrelación Espacial

Este notebook profundiza en el análisis espacial de la densidad de amenidades urbanas. El objetivo es determinar si la distribución de servicios es aleatoria o si presenta patrones de agrupamiento (clusters) significativos.

Utilizamos métricas de **Autocorrelación Espacial**:
1. **Moran's I Global:** Mide el grado de agrupamiento general en toda la comuna.
2. **LISA (Local Indicators of Spatial Association):** Identifica la ubicación exacta de Hotspots y Coldspots.

---

In [1]:
import geopandas as gpd
import libpysal
from esda.moran import Moran, Moran_Local
from splot.esda import moran_scatterplot, lisa_cluster
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE CONEXIÓN ---
load_dotenv("../.env")
db_url = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST', 'localhost')}:5432/{os.getenv('POSTGRES_DB')}"
engine = create_engine(db_url)

print("Librerías geoestadísticas cargadas.")

## 1. Carga de Datos Agregados

Para el análisis geoestadístico, necesitamos datos agregados en una grilla regular o hexagonal. Esto permite comparar una celda con sus vecinas.

In [2]:
# --- LECTURA DE LA GRILLA DE AMENIDADES ---
# 'amenity_clusters' es el resultado de un join espacial previo (puntos -> rectángulos/hexágonos)
grid = gpd.read_postgis("SELECT * FROM raw_data.amenity_clusters", engine, geom_col='geometry')

# Aseguramos un sistema de coordenadas proyectado (UTM) para cálculos de distancia precisos
if grid.crs.is_geographic:
    grid = grid.to_crs(epsg=32719) # UTM 19S para Chile Central

print(f"Análisis sobre {len(grid)} unidades espaciales.")
grid.plot(column='count', cmap='OrRd', legend=True, figsize=(8, 8))
plt.title("Mapa Coroplético: Densidad de Servicios por Celda")
plt.show()

## 2. Definición de Vecindad (Pesos Espaciales)

Antes de calcular Moran's I, debemos definir qué significa 'ser vecino'. Usaremos el criterio de **Queen Contiguity** (vecinos que comparten aristas o vértices).

In [3]:
# --- MATRIZ DE PESOS (W) ---
w = libpysal.weights.Queen.from_dataframe(grid)
w.transform = 'r' # Normalización por filas para interpretación probabilística

# --- MORAN'S I GLOBAL ---
y = grid['count'].fillna(0)
moran_global = Moran(y, w)

print(f"Índice de Moran Global: {moran_global.I:.4f}")
print(f"P-valor: {moran_global.p_sim:.4f}")

if moran_global.p_sim < 0.05:
    print("Resultado: Existe un agrupamiento espacial estadísticamente significativo.")
else:
    print("Resultado: La distribución es aleatoria.")

## 3. Análisis Local (LISA)

El análisis LISA nos permite clasificar cada celda en:
- **High-High (HH):** Una celda con alta densidad rodeada de vecinas con alta densidad (**Hotspot**).
- **Low-Low (LL):** Una celda con baja densidad rodeada de vecinas con baja densidad (**Coldspot**).
- **Atípicos (HL/LH):** Celdas que difieren drásticamente de su entorno.

In [4]:
# --- CÁLCULO DE MORAN LOCAL ---
moran_loc = Moran_Local(y, w)

# --- VISUALIZACIÓN DE CLUSTERS ---
fig, ax = plt.subplots(figsize=(10, 8))
lisa_cluster(moran_loc, grid, p=0.05, ax=ax)
plt.title("Mapa de Clusters LISA: Segregación Espacial de Servicios")
plt.show()

# --- PERSISTENCIA ---
# Guardamos el tipo de cluster para usarlo en la Síntesis y la App
grid['cluster_type'] = moran_loc.q # Cuadrantes (1:HH, 2:LH, 3:LL, 4:HL)
# Solo marcamos si el p-valor es significativo
grid['significant'] = moran_loc.p_sim < 0.05

grid.to_postgis("lisa_results", engine, schema='raw_data', if_exists='replace')
print("Resultados de Moran Local exportados a la base de datos.")